In [37]:
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
import json
from utils import write_jsonl


In [39]:
def calculate_bleu(candidate, reference):
    '''
    candidate, reference: generated and ground-truth sentences
    '''
    weights = [
         (1./2., 1./2.),
         (1./3., 1./3., 1./3.),
         (1./4., 1./4., 1./4., 1./4.)
    ]
    reference = reference.strip().split()
    candidate = candidate.strip().split()
    score = sentence_bleu([reference], candidate, weights=weights)
    return score

In [22]:
llama_output_path = '/Users/raunaksood/Desktop/RAG/deepseek7B_results_bleu.jsonl'
save_path = '/Users/raunaksood/Desktop/RAG/deepseek7B_results_bleu_NEW.jsonl'
with open(llama_output_path) as f:
    llama_outputs = [json.loads(line) for line in f]

responses = [llama_outputs[i]['response'] for i in range(len(llama_outputs))]
answers = [llama_outputs[i]['answer'] for i in range(len(llama_outputs))]
questions = [llama_outputs[i]['question'] for i in range(len(llama_outputs))]
bleus = [llama_outputs[i]['bleu'] for i in range(len(llama_outputs))]

res = []
for i in range(len(responses)):
    question = questions[i]
    response = responses[i]
    answer = answers[i]
    bleu = bleus[i]
    match = response == answer
    res.append({'question' : question, 'response' : response, 'answer' : answer, 'bleu' : bleu, 'match' : match})

    
write_jsonl(save_path, res)

# F-1 score

In [ ]:
# following this paper: SQuAD: 100,000+ Questions for Machine Comprehension of Text
#  P = num of correct tokens in predicted answer/ total tokens in predicted anser
#  R = num of correct tokens in predicted answer/ total tokens in correct answer
# F1 score = 2 * (P*R)/(P+R)

# Approach 1 - sliding window (lower f1 score) - DONE
# Approach 2 - logistic regression (higher f1 score)


In [52]:
# Approach 1 - sliding window (lower f1 score)
import json
import re
import os

def tokenize(text):
    text = text.lower()
    text = re.sub(r'(\w)-(\w)', r'\1\2', text)  
    text = re.sub(r'[^a-z0-9\s]', '', text)  
    tokens = text.split()
    return tokens


def f1_score_per_question(true_ans, pred_ans):
    common = len(set(true_ans) & set(pred_ans))
    P = common / len(pred_ans) if len(pred_ans) > 0 else 0  
    R = common / len(true_ans) if len(true_ans) > 0 else 0  
    if P + R == 0:
        return 0, R, common, len(pred_ans), len(true_ans)  
    f1_score = 2 * (P * R) / (P + R)
    return f1_score, R, common, len(pred_ans), len(true_ans)

def calculate_f1_for_file(file_path):
    total_f1_score = 0
    total_recall = 0
    num_questions = 0

    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            correct_answer = data['answer']
            predicted_response = data['response']
            correct_answer_tokens = tokenize(correct_answer)
            response_tokens = tokenize(predicted_response)
            f1, recall, overlap, total_predicted, total_correct = f1_score_per_question(correct_answer_tokens, response_tokens)
            total_f1_score += f1
            total_recall += recall
            num_questions += 1

    average_f1_score = total_f1_score / num_questions if num_questions > 0 else 0
    average_recall = total_recall / num_questions if num_questions > 0 else 0
    return average_f1_score, average_recall

def calculate_f1_for_files(file_paths):
    results_dict = {}
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        f1_score, recall = calculate_f1_for_file(file_path)
        results_dict[file_name] = {'F1 Score': f1_score, 'Recall': recall}
    return results_dict


file_paths = ['deepseek7B_results.jsonl', 'llama3_8B_results.jsonl', 'mistral7B_results.jsonl']  # Replace with your file paths
results = calculate_f1_for_files(file_paths)


In [53]:

for file_name, scores in results.items():
    print(f"{file_name}:")
    print(f"  F1 Score: {scores['F1 Score']:.2f}")
    print(f"  Recall: {scores['Recall']:.2f}")


deepseek7B_results.jsonl:
  F1 Score: 0.26
  Recall: 0.48
llama3_8B_results.jsonl:
  F1 Score: 0.28
  Recall: 0.59
mistral7B_results.jsonl:
  F1 Score: 0.24
  Recall: 0.40


# IAA (Grant)

In [22]:
# select random 50 questions

import json
import os
import random

def create_subset(file_path, selected_indices):
    new_data = []
    file_name = os.path.basename(file_path)
    new_file_name = file_name.replace('.jsonl', '_subset_grant_iaa.jsonl') 
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        for i in selected_indices:
            data = json.loads(lines[i])            
            new_data_entry = {
                "question": data["question"],
                "response": data["response"],
                "answer": "",   
                "bleu": [],     
                "match": None  
            }
            new_data.append(new_data_entry)

    new_file_path = os.path.join(os.getcwd(), new_file_name)
    with open(new_file_path, 'w') as new_file:
        for entry in new_data:
            new_file.write(json.dumps(entry) + '\n')
def subsets(file_paths):
    with open(file_paths[0], 'r') as file:
        lines = file.readlines()
        num_lines = len(lines)
        selected_indices =  random.sample(range(num_lines), 50)
    for file_path in file_paths:
        create_subset(file_path, selected_indices)
    return selected_indices
files = ['deepseek7B_results.jsonl', 'llama3_8B_results.jsonl', 'mistral7B_results.jsonl']  
selected_indices = subsets(files)


In [36]:
answers =  ["East Carson Street",
            "Gene Kelly",
            "1758",
            "Forbes Field",
            "Monongahela Incline Funicular, the T",
            "In 1982",
            "Primanti Brothers",
            "My heart is in the work",
            "2016",
            "professor Scott Fahlman",
            "The Kraft Heinz Company",
            "William Pitt, 1st Earl of Chatham",
            "Pittsburghese, or Yinzer",
            "Bloomfield",
            "Anti-Flag",
            "General John Forbes",
            "MJ the Musical",
            "PNC",
            "Buggy",
            "Kennywood",
            "Edward Manning Bigelow",
            "Pirates or Pittsburgh Pirates",
            "1979", 
            "Linda Babcock",
            "7th best Drama School",
            "The Pitt News or The Pitt Weekly",
            "$7 billion",
            "Pittsburgh Irish Festival",
            "November 19th, 2024",
            "In 1991",
            "WYEP Summer Music Festival, Pittsburgh International Jazz Festival",
            "Scotty or Scottish Terrier",
            "The Mellon Institute",
            "1979",
            "Smithfield Street Bridge",
            "#2 in undergraduate Computer Science",
            "Point Breeze",
            "Mister Rogers' Neighborhood",
            "City of bridges",
            "In 1983",
            "After Roberto Clemente Bridge",
            "Heinz Hall for the Performing Arts",
            "Pennsylvania German",
            "Fred Rogers",
            "Carnegie Steel Company",
            "McKees Rocks Bridge",
            "The Baltimore Ravens",
            "New York Giants",
            "April 3rd - 5th 2025",
            "Boston"]     

50


In [40]:
def clean_text(text):
    text = text.strip().lower()  
    text = re.sub(r'[^a-z0-9]', '', text)  
    return text

def update_answers_in_files(file_paths, answers):
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            lines = file.readlines() 

        updated_data = []        
        for i, line in enumerate(lines):
            data = json.loads(line)
            answer = answers[i]
            data['answer'] = answer
            response = data['response']
            bleu_score = calculate_bleu(response, answer)
            data['bleu'] = bleu_score
            
            cleaned_response = clean_text(response)
            cleaned_answer = clean_text(answer)
            data['match'] = cleaned_response == cleaned_answer
            
            updated_data.append(data)
        with open(file_path, 'w') as file:
            for entry in updated_data:
                file.write(json.dumps(entry) + '\n')

files = ['deepseek7B_results_subset_grant_iaa.jsonl', 'llama3_8B_results_subset_grant_iaa.jsonl', 'mistral7B_results_subset_grant_iaa.jsonl'] 
update_answers_in_files(files, answers)

/Users/gman/anaconda3/envs/tensorflow/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/gman/anaconda3/envs/tensorflow/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/gman/anaconda3/envs/tensorflow/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain

In [55]:
files = ['deepseek7B_results_subset_grant_iaa.jsonl', 'llama3_8B_results_subset_grant_iaa.jsonl', 'mistral7B_results_subset_grant_iaa.jsonl'] 
results_iaa_grant = calculate_f1_for_files(files)
for file_name, scores in results_iaa_grant.items():
    print(f"{file_name}:")
    print(f"  F1 Score: {scores['F1 Score']:.2f}")
    print(f"  Recall: {scores['Recall']:.2f}")


deepseek7B_results_subset_grant_iaa.jsonl:
  F1 Score: 0.12
  Recall: 0.41
llama3_8B_results_subset_grant_iaa.jsonl:
  F1 Score: 0.14
  Recall: 0.45
mistral7B_results_subset_grant_iaa.jsonl:
  F1 Score: 0.13
  Recall: 0.32


# Now evaluae metrics for RAG results

In [45]:
# add those fields:
# "answer": "",   
# "bleu": [],     
# "match": None 
import json

def add_fields(file_path):
    with open(file_path, 'r') as infile:
        lines = infile.readlines()
    with open(file_path, 'w') as outfile:
        for line in lines:
            data = json.loads(line)
            data["answer"] = ""
            data["bleu"] = []
            data["match"] = None
            outfile.write(json.dumps(data) + '\n')

file_path = 'mistral7B_results_with_rag.jsonl' 
add_fields(file_path)

In [48]:
# now copy ground truth and compute bleu and exact match
def copy_ans_compute_metrics(source_file, target_file):
   
    with open(source_file, 'r') as source_f, open(target_file, 'r') as target_f:
        source_lines = source_f.readlines()  
        target_lines = target_f.readlines()  
    
   
    updated_data = []
    for index, (source_line, target_line) in enumerate(zip(source_lines, target_lines)):
        source_data = json.loads(source_line)
        target_data = json.loads(target_line)
        answer = source_data["answer"]
        target_data["answer"] = answer
        response = target_data['response']  
        bleu_score = calculate_bleu(response, answer)
        target_data['bleu'] = bleu_score
        cleaned_response = clean_text(response)
        cleaned_answer = clean_text(answer)
        target_data['match'] = cleaned_response == cleaned_answer
        updated_data.append(json.dumps(target_data))

    with open(target_file, 'w') as target_f:
        for line in updated_data:
            target_f.write(line + '\n')

    print(f"Updated {target_file} with answers, BLEU scores, and match results.")


source_file = 'mistral7B_results.jsonl'  
target_file = 'mistral7B_results_with_rag.jsonl'  
copy_ans_compute_metrics(source_file, target_file)




Updated mistral7B_results_with_rag.jsonl with answers, BLEU scores, and match results.


/Users/gman/anaconda3/envs/tensorflow/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/gman/anaconda3/envs/tensorflow/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/gman/anaconda3/envs/tensorflow/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain

In [56]:
results_iaa_grant = calculate_f1_for_files([target_file])
for file_name, scores in results_iaa_grant.items():
    print(f"{file_name}:")
    print(f"  F1 Score: {scores['F1 Score']:.2f}")
    print(f"  Recall: {scores['Recall']:.2f}")

mistral7B_results_with_rag.jsonl:
  F1 Score: 0.41
  Recall: 0.52
